In [1]:
import pandas as pd
df = pd.read_csv('dataProcessed.csv').dropna()
df.reset_index(drop=True, inplace=True)

In [2]:
MAXBODYSIZE = 300
MAXHEADSIZE = 100
EMBEDDINGDIM = 300

Stances = {'agree', 'disgree', 'discuss', 'unrelated'}

In [3]:
# lol bye memory :,( 
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec

model = KeyedVectors.load_word2vec_format('GensimVectors/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
df.head(5)

,Body ID,articleBody,Headline,Stance,headProcessed,bodyProcessed
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,soldier shoot parliament lock down after gunfi...,a small meteorit crash into a wood area in nic...
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,tourist dub spider man after spider burrow und...,a small meteorit crash into a wood area in nic...
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,luke somer in fail rescu attempt in yemen,a small meteorit crash into a wood area in nic...
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,break soldier shoot at war memori in ottawa,a small meteorit crash into a wood area in nic...
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,giant catfish weigh stone catch in itali be th...,a small meteorit crash into a wood area in nic...


In [5]:
    
# unique bodies
Bodydf = df.groupby('Body ID').first()[['bodyProcessed']]
Bodydf.reset_index(inplace=True)
print(Bodydf.head(5))

# unique Heads
Headdf = df[['Body ID','headProcessed','Stance']]
print(Headdf.head(5))

   Body ID                                      bodyProcessed
0        0  a small meteorit crash into a wood area in nic...
1        4  last week we hint at what wa to come as ebola ...
2        5  newser wonder how long a quarter pounder with ...
3        6  post photo of a child onlin isi support announ...
4        7  at least suspect boko haram insurg be kill in ...
   Body ID                                      headProcessed     Stance
0        0  soldier shoot parliament lock down after gunfi...  unrelated
1        0  tourist dub spider man after spider burrow und...  unrelated
2        0          luke somer in fail rescu attempt in yemen  unrelated
3        0        break soldier shoot at war memori in ottawa  unrelated
4        0  giant catfish weigh stone catch in itali be th...  unrelated


In [6]:
Headdf.head(5)

,Body ID,headProcessed,Stance
0,0,soldier shoot parliament lock down after gunfi...,unrelated
1,0,tourist dub spider man after spider burrow und...,unrelated
2,0,luke somer in fail rescu attempt in yemen,unrelated
3,0,break soldier shoot at war memori in ottawa,unrelated
4,0,giant catfish weigh stone catch in itali be th...,unrelated


In [7]:
''' set maximum dimension '''
colIndex = Bodydf.columns.get_loc("bodyProcessed")
for index,row in Bodydf.iterrows():
    Bodydf.iat[index, colIndex] = " ".join([word for word in row['bodyProcessed'].split(' ') if word in model][:MAXBODYSIZE])
Bodydf.to_csv('TrainBodyEmbed.csv',index=False)
    
colIndex = Headdf.columns.get_loc("headProcessed")
for index,row in Headdf.iterrows():
    Headdf.iat[index, colIndex] = " ".join([word for word in row['headProcessed'].split(' ') if word in model][:MAXHEADSIZE])
Headdf.to_csv('TrainHeadEmbed.csv',index=False)


In [8]:
totalText = []

In [9]:
def totalWordsForEmbedding(dfHead, dfBody):
    global totalText
    for index, row in dfHead.iterrows():
        totalText.append(row['headProcessed'].split(' '))
    for index, row in dfBody.iterrows():
        totalText.append(row['bodyProcessed'].split(' '))

In [10]:
totalWordsForEmbedding(Headdf, Bodydf)

In [11]:
''' CONDENSED FOR TEST '''

df = pd.read_csv('testProcessed.csv').dropna()
df.reset_index(drop=True, inplace=True)
Bodydf = df.groupby('Body ID').first()[['bodyProcessed']]
Bodydf.reset_index(inplace=True)
print(Bodydf.head(5))

# unique Heads
Headdf = df[['Body ID','headProcessed']]
print(Headdf.head(5))
''' set maximum dimension '''
colIndex = Bodydf.columns.get_loc("bodyProcessed")
for index,row in Bodydf.iterrows():
    Bodydf.iat[index, colIndex] = " ".join([word for word in row['bodyProcessed'].split(' ') if word in model][:MAXBODYSIZE])
Bodydf.to_csv('TestBodyEmbed.csv',index=False)
    
colIndex = Headdf.columns.get_loc("headProcessed")
for index,row in Headdf.iterrows():
    Headdf.iat[index, colIndex] = " ".join([word for word in row['headProcessed'].split(' ') if word in model][:MAXHEADSIZE])
Headdf.to_csv('TestHeadEmbed.csv',index=False)

totalWordsForEmbedding(Headdf, Bodydf)

   Body ID                                      bodyProcessed
0        1  ha deni isra report state that he offer to ext...
1        2  a bereav afghan mother take reveng on the tali...
2        3  cnbc be report tesla ha choose nevada as the s...
3       12  a version of the iphon be say to be in develop...
4       19  gr editor s note there be no report in the wes...
   Body ID                                      headProcessed
0        1        appl instal safe to protect gold watch edit
1        1  deni claim he will give sinai land to palestinian
2        1       appl to keep gold watch edit in special safe
3        1  appl store to keep gold edit appl watch in cus...
4        1  south korean woman hair by robot vacuum cleane...


In [12]:
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(totalText)
wordIndexs = tokenizer.word_index
vocabSize = tokenizer.word_counts
print(len(wordIndexs), len(vocabSize), tokenizer.document_count)

8384 8384 77970


In [14]:
wordIndexsdf = pd.DataFrame.from_dict(wordIndexs, orient='index')
wordIndexsdf.to_csv('wordIndexs.csv',index=False)
wordIndexsdf.head(10)

,0
the,1
in,2
be,3
on,4
for,5
isi,6
that,7
say,8
it,9
have,10


In [15]:
embeddingVector = {}
for word, index in wordIndexs.items():
    if word != '':
        embeddingVector[index] = model[word]
embeddingdf = pd.DataFrame.from_dict(embeddingVector, orient='index')
embeddingdf.head(10)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1,0.080078,0.104980,0.049805,0.053467,-0.067383,-0.120605,0.035156,-0.118652,0.043945,0.030151,...,-0.071289,-0.030151,-0.013000,0.016357,-0.018311,0.014832,0.005005,0.003662,0.047607,-0.068848
2,0.070312,0.086914,0.087891,0.062500,0.069336,-0.108887,-0.081543,-0.154297,0.020752,0.131836,...,-0.168945,-0.088867,-0.080566,0.064941,0.061279,-0.047363,-0.058838,-0.047607,0.014465,-0.062500
3,-0.228516,-0.088379,0.127930,0.150391,-0.073242,0.086426,0.063965,0.096680,0.058350,0.143555,...,-0.109863,0.064941,0.117188,0.045410,0.214844,0.042969,-0.139648,-0.212891,0.188477,-0.145508
4,0.026733,-0.090820,0.027832,0.204102,0.006226,-0.090332,0.022583,-0.161133,0.132812,0.061035,...,0.026855,-0.027954,0.030884,0.040527,-0.130859,0.083008,0.015747,-0.116699,-0.029419,-0.070801
5,-0.011780,-0.047363,0.044678,0.063477,-0.018188,-0.063965,-0.001312,-0.072266,0.064453,0.086426,...,-0.022583,0.003723,-0.082520,0.081543,0.007935,0.000477,0.018433,0.071289,-0.034912,0.024170
6,-0.124512,-0.042480,0.120117,0.184570,-0.132812,0.226562,-0.126953,0.049316,-0.056396,0.100098,...,-0.035400,0.091797,0.079590,-0.134766,-0.200195,-0.009583,-0.162109,-0.388672,-0.064941,0.314453
7,-0.015747,-0.028320,0.083496,0.050293,-0.110352,0.031738,-0.014221,-0.089844,0.117676,0.118164,...,-0.011292,-0.015625,-0.033447,-0.020630,-0.019409,0.063965,0.020142,0.006866,0.061035,-0.148438
8,-0.036133,-0.121094,0.133789,0.114258,-0.172852,0.082031,0.218750,-0.136719,0.173828,0.182617,...,0.015503,0.050049,0.069336,-0.015869,-0.046143,-0.265625,-0.011780,-0.086426,0.143555,0.027344
9,0.084473,-0.000353,0.053223,0.099609,-0.148438,-0.062500,0.056396,-0.064941,0.065918,0.018799,...,-0.032959,0.147461,-0.157227,0.009705,-0.136719,0.031982,0.118652,0.017090,0.060791,-0.108887
10,-0.139648,-0.034668,-0.053711,0.179688,-0.036865,-0.025757,0.004852,-0.083496,0.008179,0.324219,...,-0.049072,0.063477,0.080566,0.088867,0.077637,-0.046631,-0.071777,-0.118164,-0.002457,-0.072266


In [16]:
embeddingdf.to_csv('embeddingVectors.csv',index=False)

In [17]:
trainBodyDf = pd.read_csv('TrainBodyEmbed.csv')
trainHeadDf = pd.read_csv('TrainHeadEmbed.csv')

In [18]:
trainBodyDf.head(10)

,Body ID,bodyProcessed
0,0,small crash into wood area in capit overnight ...
1,4,last week we hint at what wa come as ebola fea...
2,5,newser wonder how long quarter pounder with ca...
3,6,post photo child onlin isi support announc tha...
4,7,at least suspect haram be kill in clash betwee...
5,8,there be so much fake stuff on the internet in...
6,9,cnn crash down in late saturday night caus lou...
7,10,move over netflix hulu word ha it that amazon ...
8,11,we ve all see the depict god as with flow mane...
9,13,soldier ha be shoot at canada s nation war wit...


In [19]:
trainHeadDf.head(5)

,Body ID,headProcessed,Stance
0,0,soldier shoot parliament lock down after erupt...,unrelated
1,0,tourist dub spider man after spider burrow und...,unrelated
2,0,luke in fail attempt in yemen,unrelated
3,0,break soldier shoot at war in ottawa,unrelated
4,0,giant catfish weigh stone catch in be think be...,unrelated


In [20]:
trainBodyDf.head(5)

,Body ID,bodyProcessed
0,0,small crash into wood area in capit overnight ...
1,4,last week we hint at what wa come as ebola fea...
2,5,newser wonder how long quarter pounder with ca...
3,6,post photo child onlin isi support announc tha...
4,7,at least suspect haram be kill in clash betwee...


In [21]:
''' Takes a minute or two '''
def CreateTrainingData(bodydf, headdf):
    heads = []
    bodies = []
    stances = []
    stancesLookup = {'unrelated': 0 , 'agree':1, 'disagree':2, 'discuss':3}
    for index, row in headdf.iterrows():
        if not pd.isna(row['headProcessed']):
            heads.append(row['headProcessed'].split(" "))
            bodies.append(bodydf.loc[bodydf['Body ID'] == int(row['Body ID'])].iloc[0]['bodyProcessed'][0].split(" "))
            stances.append(stancesLookup[row['Stance'].strip()])
    heads = tokenizer.texts_to_sequences(heads)
    bodies = tokenizer.texts_to_sequences(bodies)
    heads = pad_sequences(heads,maxlen = MAXHEADSIZE,padding = 'post')
    bodies = pad_sequences(bodies,maxlen = MAXBODYSIZE,padding = 'post')
    stances = to_categorical(stances, num_classes=4)
    return heads,bodies,stances
heads,bodies,stances = CreateTrainingData(trainBodyDf, trainHeadDf)

In [22]:
len(heads)

49952

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input, Lambda
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.layers import concatenate,dot
from keras.preprocessing import sequence
import keras.backend as K
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
import pydotplus
import pydot
from keras.utils.vis_utils import model_to_dot
import keras
keras.utils.vis_utils.pydot = pydot


In [24]:
import numpy as np
embedMatrix = np.zeros((len(tokenizer.word_counts)+1 , EMBEDDINGDIM))
for word, index in wordIndexs.items():
    if word != '':
        embedMatrix[index] = embeddingVector[tokenizer.texts_to_sequences([word])[0][0]]

In [25]:


InputHead = Input(shape=(MAXBODYSIZE,), dtype='int32', name='InputHead')
InputBody = Input(shape=(MAXHEADSIZE,), dtype='int32', name='InputBody')
Embeddings = Embedding(len(wordIndexs) + 1, EMBEDDINGDIM, weights=[embedMatrix],trainable=False)
EmbedHead = Embeddings(InputHead)
EmbedBody = Embeddings(InputBody)



LSTMHead = Bidirectional(LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='LSTMHead'))(EmbedHead)
LSTMBody = Bidirectional(LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='LSTMBody'))(EmbedBody)

DotLayer = dot([LSTMHead,LSTMBody],axes = 1, normalize=True)

Concat = concatenate([LSTMHead,LSTMBody,DotLayer])

DenseLayer = Dense(100,activation='relu')(Concat)
DenseLayer = Dropout(0.3)(DenseLayer)
DenseLayer = Dense(4,activation='softmax')(DenseLayer)
LSTMNetwork = Model(inputs=[InputHead,InputBody], outputs=[DenseLayer])
LSTMNetwork.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])
print(LSTMNetwork.summary())


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
InputHead (InputLayer)          (None, 300)          0                                            
__________________________________________________________________________________________________
InputBody (InputLayer)          (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         multiple             2515500     InputHead[0][0]                  
                                                                 InputBody[0][0]                  
_____________________

In [26]:
''' CANT GET TO WORK? shows network graph, currently a keras bug 
plot_model(LSTMNetwork, to_file='LSTMNetworkMap.png')
SVG(model_to_dot(LSTMNetwork).create(prog='dot', format='svg'))

If you are getting errors here for pydot install graphviz & pydot & pydotplus
GraphViz needs to be installed using apt-get / brew
otherwise comment out sainvg\

'''

" CANT GET TO WORK? shows network graph, currently a keras bug \nplot_model(LSTMNetwork, to_file='LSTMNetworkMap.png')\nSVG(model_to_dot(LSTMNetwork).create(prog='dot', format='svg'))\n\nIf you are getting errors here for pydot install graphviz & pydot & pydotplus\nGraphViz needs to be installed using apt-get / brew\notherwise comment out sainvg\n"

In [27]:
assert(len(heads) == len(stances))
splitNum = int(len(heads) * 2 / 3)
splitNum

33301

In [28]:
TrainHeadSplit, TrainBodySplit, TrainStanceSplit = bodies[:splitNum], heads[:splitNum], stances[:splitNum]
TestHeadSplit, TestBodySplit, TestStanceSplit = bodies[splitNum:], heads[splitNum:], stances[splitNum:]


# validation_data = ([TestHeadSplit, TestBodySplit],[TestStanceSplit])
for i in range(10):
    LSTMNetwork.fit([TrainHeadSplit, TrainBodySplit],[TrainStanceSplit], validation_data = ([TestHeadSplit, TestBodySplit],[TestStanceSplit]), epochs=4, batch_size=128,verbose = True)

Instructions for updating:
Use tf.cast instead.
Train on 33301 samples, validate on 16651 samples
Epoch 1/4
33301/33301 [==============================] - 653s 20ms/step - loss: 0.7185 - acc: 0.7740 - val_loss: 0.9485 - val_acc: 0.6358
Epoch 2/4
33301/33301 [==============================] - 648s 19ms/step - loss: 0.6630 - acc: 0.7792 - val_loss: 0.9634 - val_acc: 0.6358
Epoch 3/4
33301/33301 [==============================] - 648s 19ms/step - loss: 0.6396 - acc: 0.7791 - val_loss: 0.9868 - val_acc: 0.6353
Epoch 4/4
33301/33301 [==============================] - 647s 19ms/step - loss: 0.6107 - acc: 0.7810 - val_loss: 0.9703 - val_acc: 0.6360
Train on 33301 samples, validate on 16651 samples
Epoch 1/4
33301/33301 [==============================] - 647s 19ms/step - loss: 0.5780 - acc: 0.7856 - val_loss: 1.0282 - val_acc: 0.6388
Epoch 2/4
33301/33301 [==============================] - 648s 19ms/step - loss: 0.5487 - acc: 0.7938 - val_loss: 0.9537 - val_acc: 0.6389
Epoch 3/4
33301/33301 [=

In [29]:
LSTMNetwork.save('model.5h')